<a href="https://colab.research.google.com/github/jieun-seong/MRC_redistricting/blob/main/py38_success_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget -O mini.sh https://repo.anaconda.com/miniconda/Miniconda3-py38_4.8.2-Linux-x86_64.sh
!chmod +x mini.sh
!bash ./mini.sh -b -f -p /usr/local
!conda install -q -y jupyter
!conda install -q -y google-colab -c conda-forge
!python -m ipykernel install --name "py38" --user

--2022-06-14 14:52:20--  https://repo.anaconda.com/miniconda/Miniconda3-py38_4.8.2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 89817099 (86M) [application/x-sh]
Saving to: ‘mini.sh’

mini.sh             100%[===================>]  85.66M  71.5MB/s    in 1.2s    

2022-06-14 14:52:21 (71.5 MB/s) - ‘mini.sh’ saved [89817099/89817099]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: / - done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - asn1crypto==1.3.0=py38_0
    - ca-certificates==2020.1.1=0
    - certifi==2019.11.28=py38_0
    - cffi==1.14.0=py38h2e261b9_0
    - chardet==3.0.4=py38_1003
    - conda-package-handling==1.6.0=py38h7b6447c_0
    - conda==4.8.2=py38_0
    - cryptography==2.8

In [1]:
# Reload the web page and execute this cell
import sys
print("User Current Version:-", sys.version)

User Current Version:- 3.8.1 (default, Jan  8 2020, 22:29:32) 
[GCC 7.3.0]


In [2]:
!pip install gerrychain

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 56 kB 2.0 MB/s 
     |████████████████████████████████| 2.0 MB 6.6 MB/s 
     |████████████████████████████████| 1.0 MB 25.0 MB/s 
     |████████████████████████████████| 41.6 MB 1.2 MB/s 
     |████████████████████████████████| 11.3 MB 68.4 MB/s 
     |████████████████████████████████| 2.1 MB 63.1 MB/s 
     |████████████████████████████████| 7.7 MB 35.1 MB/s 
     |████████████████████████████████| 16.6 MB 25 kB/s 
     |████████████████████████████████| 1.2 MB 60.1 MB/s 
     |████████████████████████████████| 3.1 MB 55.4 MB/s 
     |████████████████████████████████| 930 kB 56.6 MB/s 
     |████████████████████████████████| 96 kB 7.6 MB/s 


In [3]:
import matplotlib.pyplot as plt
from gerrychain import (GeographicPartition, Partition, Graph, MarkovChain,
                        proposals, updaters, constraints, accept, Election)
from gerrychain.proposals import recom
from functools import partial
import pandas, tqdm
import geopandas as gpd

# this makes the random assignment
from gerrychain.tree import recursive_tree_part

In [4]:
party_to_favor = "Democratic" #"Republican"
election_name = 'PRES16' 
bias_measure = 'safe_seats'
bias_measures = ['safe_seats', 'efficiency_gap', 'mean_median', 'partisan_bias', 'partisan_gini']

In [5]:
import numpy as np

def get_biased_accept(bias_measure):
    accept_dict = {
        'safe_seats' : increase_safe_seats,
        'efficiency_gap' : increase_efficiency_gap,
        'mean_median' : increase_mean_median,
        'partisan_bias' : increase_partisan_bias,
        'partisan_gini' : increase_partisan_gini
    }

    return accept_dict[bias_measure]

# this is from the Duchin, Needham, Weighill paper
def increase_safe_seats(partition):
    parent = partition.parent
    DEMseats = len(
        [x for x in partition[election_name].percents(party_to_favor) if x > 0.53]
    )
    DEMseatsparent = len(
        [x for x in parent[election_name].percents(party_to_favor) if x > 0.53]
    )
    alpha = np.exp(2*(DEMseats-DEMseatsparent))
    doaccept = (np.random.random() < alpha)
    return doaccept

def get_elec_results(x):
    # get ElectionResults for a map
    return x.updaters[election_name](x)

def increase_efficiency_gap(partition):
    parent = partition.parent

    curr = get_elec_results(partition)
    prev = get_elec_results(parent)

    ## change here if we are interested in different "bias" metric
    # https://gerrychain.readthedocs.io/en/latest/api.html#module-gerrychain.metrics
    delta = curr.efficiency_gap() - prev.efficiency_gap()
    beta = 300
        
    # accept immediately if "bias" is not decreasing
    # else: accept with some probability, Metropolis style
    return np.random.random() < np.exp(beta*delta)

def increase_mean_median(partition):
    parent = partition.parent

    curr = get_elec_results(partition)
    prev = get_elec_results(parent)

    delta = curr.mean_median() - prev.mean_median()
    beta = 300 # this should change with the bias metric
    
    # accept immediately if "bias" is not decreasing
    # else: accept with some probability, Metropolis style
    return np.random.random() < np.exp(beta*delta)

def increase_partisan_bias(partition):
    parent = partition.parent

    curr = get_elec_results(partition)
    prev = get_elec_results(parent)

    delta = curr.partisan_bias() - prev.partisan_bias()
    beta = 50 # this should change with the bias metric
        
    # accept immediately if "bias" is not decreasing
    # else: accept with some probability, Metropolis style
    return np.random.random() < np.exp(beta*delta)


def increase_partisan_gini(partition):
    parent = partition.parent

    curr = get_elec_results(partition)
    prev = get_elec_results(parent)

    delta = curr.partisan_gini() - prev.partisan_gini()
    beta = 300 # this should change with the bias metric
        
    # accept immediately if "bias" is not decreasing
    # else: accept with some probability, Metropolis style
    return np.random.random() < np.exp(beta*delta)


In [7]:
graph = Graph.from_json("NC.json")

In [12]:
elections = [
    # Election("SEN12", {"Democratic": "SEN12D", "Republican": "SEN12R"}),
    # Election("SEN16", {"Democratic": "EL16G_US_1", "Republican": "EL16G_USS_"}),
    # Election("PRES12", {"Democratic": "PRES12D", "Republican": "PRES12R"}),
    Election("PRES16", {"Democratic": "EL16G_PR_D", "Republican": "EL16G_PR_R"})
]

# Population updater, for computing how close to equality the district
# populations are. "TOTPOP" is the population column from our shapefile.
my_updaters = {"population": updaters.Tally("TOTPOP", alias="population")}

# Election updaters, for computing election results using the vote totals
# from our shapefile.
election_updaters = {election.name: election for election in elections}
my_updaters.update(election_updaters)

initial_partition = GeographicPartition(graph, assignment="CD", updaters=my_updaters)

# find ideal population per district
ideal_pop = sum(initial_partition["population"].values()) / len(initial_partition)

# create a random assignment
random_assignment = recursive_tree_part(graph, 
                    # how many districts
                    parts = range(13),
                    # what is the population target
                   pop_target = ideal_pop,
                    # what column contains population infor
                    pop_col = "TOTPOP",
                    # allowed variation
                    epsilon = .01,
                    # still don't know what this is
                    node_repeats = 2
                   )

# create the random parititon by giving it the random assignment
random_partition = Partition(graph,
                              assignment = random_assignment,
                              updaters = my_updaters)
# The ReCom proposal needs to know the ideal population for the districts so that
# we can improve speed by bailing early on unbalanced partitions.

ideal_population = sum(initial_partition["population"].values()) / len(initial_partition)

# We use functools.partial to bind the extra parameters (pop_col, pop_target, epsilon, node_repeats)
# of the recom proposal.
proposal = partial(recom,
                   pop_col="TOTPOP",
                   pop_target=ideal_population,
                   epsilon=0.02,
                   node_repeats=2
                  )
compactness_bound = constraints.UpperBound(
    lambda p: len(p["cut_edges"]),
    2*len(initial_partition["cut_edges"])
)

pop_constraint = constraints.within_percent_of_ideal_population(initial_partition, 0.02)

In [13]:
bias_chains = []
for bias in bias_measures:
    bias_chain = MarkovChain(proposal = proposal,
                            constraints = [pop_constraint, compactness_bound],
                            accept = get_biased_accept(bias),
                            initial_state = random_partition,
                            total_steps = 10000) 
    bias_chains.append(bias_chain)

chain = MarkovChain(
    proposal=proposal,
    constraints=[
        pop_constraint,
        compactness_bound
    ],
    accept=accept.always_accept,
    initial_state=random_partition,
    total_steps=10000
)



In [ ]:
bias_ensembles = [[p for p in bc.with_progress_bar()][::10] for bc in bias_chains]